<a href="https://colab.research.google.com/github/Xen-J/CanutoZero/blob/main/Empirical_Validation_of_the_UAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#About the experiment
Shallow neural network with varying number of neurons, activation functions (tanh, relu and sigmoid) and target functions (sin, gaussian, sin_high_freq).


## Exporting the data in a full-results .csv file


In [ ]:
import csv

def save_results(all_results, filename="full_results.csv"):
    """
    It creates a CSV where each row has the following structure:
    target_function, act_function, n_neurons, ...
    """
    rows = []

    # 1) Iterate through the all_results.items
    for (func, act), (results_dict, metrics) in all_results.items():
        for n, res in results_dict.items():

            row = {
                "target_function": func,
                "act_function": act,
                "n_neurons": n,
            }

            for key, value in res.items():
                if key!="losses" and key!="final_pred":
                    row[key] = value

            rows.append(row)

    # 2) Columns
    all_keys = set()
    for r in rows:
        all_keys.update(r.keys())
    all_keys = list(all_keys)

    # 3) Write CSV
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=all_keys)
        writer.writeheader()
        writer.writerows(rows)

    print(f"Saved: {filename}")


##Core module

In [ ]:
"""
Core Module for UAT Validation Experiments
===========================================
Code for validating the Universal Approximation Theorem empirically.
It defines the experiment configuration, neural network architecture,
data generation, training routines, metrics computation, and visualization.
To run RunAll.py, import this module.

Authors: Xendra Jaime Reyes, Queen-Aset Blissett
Date: November 2025
"""

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
import time
from pathlib import Path


# ============================================================================
# CONFIGURATION: ExperimentConfig will define the experiment parameters,
# such as neuron counts, activation functions, target functions, training epochs,
# learning rates, domain boundaries, convergence thresholds and others.
# ============================================================================

@dataclass
class ExperimentConfig:
    """
    Configuration for UAT validation experiments.

    Attributes:
        neuron_counts: List of hidden layer widths to test
        activation: Activation function ('relu', 'tanh', 'sigmoid')
        target_function: Function to approximate ('sin', 'sin_high_freq', 'gaussian')
        epochs: Maximum training epochs (we'll use 1000-epoch batches)
        learning_rate: Adam optimizer learning rate
        x_min, x_max: Domain boundaries
        n_samples: Number of training points
        threshold: MSE convergence threshold
        random_seed: For reproducibility
        save_dir: Directory to save results
        verbose: Print training progress
    """

    #-----We define default values-----#
    # Network architecture
    neuron_counts: List[int] = field(default_factory=lambda: [10, 50, 100, 500, 1000])
    activation: str = 'relu'

    target_function: str = 'sin'    #target function to approximate
    threshold: float = 0.01         # MSE convergence threshold
    random_seed: int = 42           # Random seed for reproducibility

    # Training hyperparameters
    epochs: int = 5000
    learning_rate: float = 0.01

    # Domain
    x_min: float = -3.0
    x_max: float = 3.0
    n_samples: int = 1000

    # Output
    save_dir: str = 'results'
    verbose: bool = True

    def __post_init__(self):
        """Validate configuration."""
        valid_activations = ['relu', 'tanh', 'sigmoid']
        if self.activation not in valid_activations:
            raise ValueError(f"activation must be one of {valid_activations}")

        valid_functions = ['sin', 'sin_high_freq', 'gaussian', 'smooth_stair']
        if self.target_function not in valid_functions:
            raise ValueError(f"target_function must be one of {valid_functions}")

# ============================================================================
# NEURAL NETWORK: To prove the UAT, we will use only a Single Hidden Layer
# architecture (SingleHiddenLayerNet) with personalised number of neurons
# and activation function.
# ============================================================================

class SingleHiddenLayerNet(nn.Module):
    """
    Single hidden layer neural network for function approximation.

    Architecture:
        Input (1D) → Hidden (n_neurons) → Activation → Output (1D)

    Args:
        n_neurons: Number of hidden layer neurons
        activation: Activation function name
    """

    def __init__(self, n_neurons: int, activation: str = 'relu'):
        super().__init__()
        self.hidden = nn.Linear(1, n_neurons)
        self.output = nn.Linear(n_neurons, 1)
        self.activation_name = activation

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.

        Args:
            x: Input tensor of shape (batch_size, 1)

        Returns:
            Output tensor of shape (batch_size, 1)
        """
        h = self.hidden(x)

        # Apply activation, we are only working with 3 types
        if self.activation_name == 'relu':
            h = torch.relu(h)
        elif self.activation_name == 'tanh':
            h = torch.tanh(h)
        elif self.activation_name == 'sigmoid':
            h = torch.sigmoid(h)

        y = self.output(h)
        return y

    def count_parameters(self) -> int:
        """Count total trainable parameters."""
        return sum(p.numel() for p in self.parameters())


# ============================================================================
# DATA GENERATION: Depending on the target function, we will generate training
# data and a suitable label for visualization.
# ============================================================================

def generate_target_data(config: ExperimentConfig) -> Tuple[torch.Tensor, torch.Tensor, str]:
    """
    Generate training data and the full dense domain set for plotting/testing.
    Args:
        config: Experiment configuration

    Returns:
        x_train: Input tensor of shape (n_samples, 1)
        y_train: Target output tensor of shape (n_samples, 1)
        x_test: Test input tensor of shape (n_samples, 1)
        y_test: Test output tensor of shape (n_samples, 1)
        label: Human-readable function label
    """
    #We generate a dense set
    x_dense = torch.linspace(config.x_min, config.x_max, config.n_samples * 2).reshape(-1, 1)

    if config.target_function == 'sin':
        y_dense = torch.sin(np.pi * x_dense)
        label = 'sin(x)'
    elif config.target_function == 'sin_high_freq':
        y_dense = torch.sin(2 * np.pi * x_dense)
        label = 'sin(2πx)'
    elif config.target_function == 'gaussian':
        y_dense = torch.exp(-x_dense**2)
        label = 'exp(-x²)'
    elif config.target_function == 'composite':
        y_dense = torch.sin(2 * np.pi * x_dense) * torch.exp(-0.3 * x_dense**2)
        label = 'sin(2πx)·exp(-0.3x²)'

    #Random index to ensure train/test split
    idxs = torch.randperm(len(x_dense))
    n_train = config.n_samples

    train_idxs = idxs[:n_train]

    x_train = x_dense[train_idxs]
    y_train = y_dense[train_idxs]

    return x_train, y_train, x_dense, y_dense, label


# ============================================================================
# TRAINING: According to the neural network built architecture, we will train
# the model using Adam optimizer, returning a dictionary with training results,
# such as losses, final predictions, MSE, MAE, max error, convergence epoch,
# training time, and convergence status.
# ============================================================================

def train_network(
    model: nn.Module,
    x_train: torch.Tensor,
    y_train: torch.Tensor,
    config: ExperimentConfig
) -> Dict:
    """
    Train neural network to approximate target function.

    Args:
        model: Neural network to train
        x_train: Training inputs
        y_train: Training targets
        config: Training configuration

    Returns:
        Tuple[model, Dict]: The trained model and dictionary of partial results.
        Dict dictionary contains:
            - losses: List of training losses per epoch
            - final_pred: Final predictions on training set
            - final_mse: Final mean squared error
            - final_mae: Final mean absolute error
            - max_error: Maximum pointwise error
            - convergence_time: seconds since start time when MSE < threshold (or None)
            - training_time: Wall-clock training time in seconds
            - converged: Boolean indicating convergence
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    losses = []
    convergence_time = 0
    start_time = time.time()

    for epoch in range(config.epochs):
        # Forward pass
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item()) # Record loss

        # Check convergence
        if loss.item() < config.threshold and convergence_time ==0:
            convergence_time = time.time() - start_time

        # Verbose output
        if config.verbose and (epoch + 1) % 1000 == 0:
            print(f"      Epoch {epoch+1}/{config.epochs}: Loss = {loss.item():.6f}")

    training_time = time.time() - start_time

    # Final evaluation
    model.eval()
    with torch.no_grad():
        final_pred = model(x_train)
        final_mse_train = criterion(final_pred, y_train).item()

    return model, {
        'losses': losses,
        'convergence_time': convergence_time if convergence_time else -1,
        'training_time': training_time,
        'converged': final_mse_train < config.threshold,
  }

# ============================================================================
# METRICS: Taking the experimental results, compute_metrics will return
# a metric summary dictionary, including minimum viable width (N_min),
# marginal efficiency, best MSE and corresponding network width.
# This way, we can compare Neuron efficiency, Convergence speed, and
# Approximation accuracy per activation function.
# ============================================================================

def compute_metrics(results: Dict, neuron_counts: List[int]) -> Dict:
    """
    Compute summary metrics from experimental results.

    Args:
        results: Dictionary mapping neuron_count → training results
        neuron_counts: List of tested network widths

    Returns:
        Dictionary containing:
            - n_min: Minimum viable width (or None if never converges)
            - marginal_efficiency: List of MSE improvement per neuron added
            - best_mse: Lowest achieved MSE
            - best_n: Network width achieving best MSE
            - fastest_n: Width with shortest training time.
            - fastest_time: Fastest training time.
            - quickest_conv_n: Network width with shortest convergence epoch.
            - quickest_conv_epoch: Shortest convergence epoch.
    """
    #--- Minimum viable width (N_min)---#
    n_min = next((n for n in neuron_counts if results[n]['converged']), None)

    #--- Marginal efficiency---#
    mse_values = [results[n]['final_mse'] for n in neuron_counts]
    marginal_eff = []
    for i in range(1, len(neuron_counts)):
        delta_n = neuron_counts[i] - neuron_counts[i-1]
        delta_mse = mse_values[i-1] - mse_values[i]
        eff = delta_mse / delta_n if delta_n > 0 else 0.0
        marginal_eff.append(eff)

    # ---Best performance (MSE and R^2) ---#
    best_idx = np.argmin(mse_values)
    best_mse = mse_values[best_idx]
    best_n = neuron_counts[best_idx]
    best_r2 = results[best_n]['final_r2'] # We use N of best MSE

    #--- Velocity (time and epochs) ---#
    times = [results[n]['training_time'] for n in neuron_counts]
    fastest_idx = np.argmin(times)
    fastest_time = times[fastest_idx]
    fastest_n = neuron_counts[fastest_idx]

    conv_epochs = [results[n]['convergence_time'] for n in neuron_counts if results[n]['converged']]
    conv_neurons = [n for n in neuron_counts if results[n]['converged']]

    quickest_conv_epoch = None
    quickest_conv_n = None

    if conv_epochs:
        quickest_idx = np.argmin(conv_epochs)
        quickest_conv_epoch = conv_epochs[quickest_idx]
        quickest_conv_n = conv_neurons[quickest_idx]


    return {
        'n_min': n_min,
        'marginal_efficiency': marginal_eff,
        'best_mse': best_mse,
        'best_r2': best_r2,
        'best_n': best_n,
        'fastest_time': fastest_time,
        'fastest_n': fastest_n,
        'quickest_conv_epoch': quickest_conv_epoch,
        'quickest_conv_n': quickest_conv_n,
    }


# ============================================================================
# TEST: We evaluate the neural network on the given test set.
# ============================================================================

def evaluate_network(model: nn.Module, x_test: torch.Tensor, y_test: torch.Tensor) -> Dict:
    """Calculte all precistion metrics on the test set."""

    model.eval()
    with torch.no_grad():
        test_pred = model(x_test)

        # 1. MSE
        final_mse = nn.MSELoss()(test_pred, y_test).item()

        # 2. MAE
        final_mae = torch.mean(torch.abs(test_pred - y_test)).item()

        # 3. Max Error
        max_error = torch.max(torch.abs(test_pred - y_test)).item()

        # 4. R^2 Score
        # RSS: Residual Sum of Squares (Error no explicado)
        RSS = torch.sum((y_test - test_pred) ** 2)
        # SST: Total Sum of Squares (Varianza total)
        SST = torch.sum((y_test - torch.mean(y_test)) ** 2)
        final_r2 = 1 - (RSS / SST).item() if SST != 0 else 0.0

    return {
        'final_pred': test_pred.numpy(), # Usamos las predicciones del test set para el gráfico
        'final_mse': final_mse,
        'final_mae': final_mae,
        'max_error': max_error,
        'final_r2': final_r2,
    }

# ============================================================================
# VISUALIZATION: Plot_experiment_results will generate comprehensive images
# of the experimental results dictionary, including a convergence plot,
# function approximation plots for every network size, error distributions
# comparing each network size, and a summary text box per image.
# ============================================================================

def plot_experiment_results(
    results: Dict,
    x: torch.Tensor,
    y_true: torch.Tensor,
    func_label: str,
    config: ExperimentConfig,
    save_path: Optional[str] = None
):
    """
    Generate comprehensive visualization of experimental results.

    Creates a figure with:
        - Convergence plot (MSE vs epoch for all network sizes)
        - Function approximation plots (one per network size)
        - Error distribution plot
        - Summary text box

    Args:
        results: Dictionary mapping neuron_count → training results
        x: Input data
        y_true: Target outputs
        func_label: Human-readable function label
        config: Experiment configuration
        save_path: Optional path to save figure
    """
    x_np = x.numpy().flatten()
    y_true_np = y_true.numpy().flatten()

    n_sizes = len(config.neuron_counts)

    # Create figure
    fig = plt.figure(figsize=(18, 10))

    # Color for this activation
    color_map = {'relu': '#1f77b4', 'tanh': '#2ca02c', 'sigmoid': '#d62728'}
    color = color_map[config.activation]

    # ========================================
    # PLOT 1 (CONVERGENCE): MSE vs Epoch for all network sizes,
    # with threshold line.
    # ========================================
    ax_conv = plt.subplot(2, n_sizes + 2, 1)

    for n in config.neuron_counts:
        losses = results[n]['losses']
        ax_conv.semilogy(losses, linewidth=2, label=f'{n}N', alpha=0.8)

    ax_conv.axhline(y=config.threshold, color='red', linestyle='--',
                   linewidth=1.5, alpha=0.7, label='Threshold')
    ax_conv.set_xlabel('Epoch', fontsize=11)
    ax_conv.set_ylabel('MSE (log scale)', fontsize=11)
    ax_conv.set_title(f'{config.activation.upper()}: Convergence',
                     fontsize=12, fontweight='bold')
    ax_conv.legend(fontsize=9, loc='upper right')
    ax_conv.grid(True, alpha=0.3)

    # ========================================
    # PLOTS 2-n (APPROXIMATIONS): One per network size,
    # plots comparing the true function and the NN prediction
    # ========================================
    for idx, n in enumerate(config.neuron_counts):
        ax = plt.subplot(2, n_sizes + 2, idx + 2)

        #---Getting predictions and metrics
        y_pred = results[n]['final_pred'].flatten()
        mse = results[n]['final_mse']
        converged = results[n]['converged']

        #---Graphing the true function and predicted function
        ax.plot(x_np, y_true_np, 'k-', linewidth=2.5,
               label=f'True: {func_label}', alpha=0.8) #True function
        ax.plot(x_np, y_pred, '--', linewidth=2.5,
               color=color, label='NN', alpha=0.8) #Predicted function
        ax.fill_between(x_np, y_true_np, y_pred, alpha=0.25, color=color) # Error area

        #---Title with status
        status = "✓" if converged else "✗"
        ax.set_title(f'{n}N: MSE={mse:.6f} {status}',
                    fontsize=11, fontweight='bold')
        ax.set_xlabel('x', fontsize=10)
        ax.set_ylabel('y', fontsize=10)
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)

    # ========================================
    # PLOT n+1 (ERROR DISTRIBUTION): Comparing absolute error
    # for every point in the domain, per network size.
    # ========================================
    ax_err = plt.subplot(2, n_sizes + 2, n_sizes + 3)

    for n in config.neuron_counts:
        y_pred = results[n]['final_pred'].flatten()
        errors = np.abs(y_true_np - y_pred)
        ax_err.plot(x_np, errors, linewidth=2, label=f'{n}N', alpha=0.8)

    ax_err.axhline(y=config.threshold, color='red', linestyle='--',
                  linewidth=1.5, alpha=0.7, label='Threshold')
    ax_err.set_xlabel('x', fontsize=11)
    ax_err.set_ylabel('|f(x) - g(x)|', fontsize=11)
    ax_err.set_title('Error Distribution', fontsize=12, fontweight='bold')
    ax_err.legend(fontsize=9)
    ax_err.grid(True, alpha=0.3)

    # ========================================
    # PLOT n+2 (SUMMARY): Text box summarizing key metrics
    # ========================================
    ax_summary_a = plt.subplot(2, n_sizes + 2, n_sizes + 4)
    ax_summary_a.axis('off')

    summary_text_a = f"{config.activation.upper()} - SUMMARY\n{'='*35}\n\n"

    for n in config.neuron_counts:
        res = results[n]
        status = "✓ CONVERGED" if res['converged'] else "✗ NOT CONVERGED"

        summary_text_a += f"{n} NEURONS: {status}\n"
        summary_text_a += f"  MSE:                 {res['final_mse']:.6f}\n"
        summary_text_a += f"  MAE:                 {res['final_mae']:.6f}\n"
        summary_text_a += f"  Max Error:           {res['max_error']:.6f}\n"
        summary_text_a += f"  Training time:       {res['training_time']:.1f}s\n"
        summary_text_a += f"  Convergence time:    {res['convergence_time']}\n\n"
        summary_text_a += f"  R^2:                 {res['final_r2']}\n\n"

    # Add metrics
    ax_summary_b = plt.subplot(2, n_sizes + 2, n_sizes + 5)
    ax_summary_b.axis('off')
    metrics = compute_metrics(results, config.neuron_counts)

    summary_text_b = f"EXPERIMENT:\n{'='*35}\n\n"
    summary_text_b += f"Learning rate:  {config.learning_rate}"
    summary_text_b += f"Threshold:      {config.threshold}"
    summary_text_b += f"Epochs:         {config.epochs}\n"

    summary_text_b += f"{'='*35}METRICS:\n{'='*35}\n\n"
    if metrics['n_min']:
        summary_text_b += f"N_min = {metrics['n_min']} neurons\n"
    else:
        summary_text_b += f"N_min > {max(config.neuron_counts)}\n"

    summary_text_b += f"Best MSE = {metrics['best_mse']:.6f}\n"
    summary_text_b += f"  at N = {metrics['best_n']} neurons\n"
    summary_text_b += f"  R^2 = {metrics['best_r2']}\n\n"

    ax_summary_a.text(0.05, 0.95, summary_text_a, transform=ax_summary_a.transAxes,
                   fontsize=9, verticalalignment='top', family='monospace',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    ax_summary_b.text(0.05, 0.95, summary_text_b, transform=ax_summary_b.transAxes,
                   fontsize=9, verticalalignment='top', family='monospace',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

    # Overall title
    fig.suptitle(f'Approximating {func_label} with {config.activation.upper()} activation',
                fontsize=14, fontweight='bold')

    plt.tight_layout(rect=[0, 0, 1, 0.98])

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Saved: {save_path}")

    plt.close()


# ============================================================================
# EXPERIMENT RUNNER: run_single_experiment will execute a complete experiment
# for a given configuration, returning the results dictionary and summary metrics.
# ============================================================================


def run_single_experiment(config: ExperimentConfig) -> Tuple[Dict, Dict]:
    """
    Run complete experiment for given configuration.

    Args:
        config: Experiment configuration

    Returns:
        results: Dictionary mapping neuron_count → training results
        metrics: Summary metrics dictionary
    """
    # Set seed
    torch.manual_seed(config.random_seed)
    np.random.seed(config.random_seed)

    if config.verbose:
        print(f"\n{'='*70}")
        print(f"EXPERIMENT: {config.target_function} + {config.activation}")
        print(f"{'='*70}")

    # Generate data
    x_train, y_train, x_plot, y_plot, label = generate_target_data(config)

    if config.verbose:
        print(f"  Function: {label}")
        print(f"  Domain: [{config.x_min}, {config.x_max}]")
        print(f"  Samples: {config.n_samples}")
        print(f"  Testing: {config.neuron_counts}")
        print(f"  Epochs: {config.epochs}, LR: {config.learning_rate}")
        print(f"  Threshold: {config.threshold}\n")
        # Print the size of the test sample
        print(f"  Train Samples: {x_train.shape[0]}, Test Samples: {x_plot.shape[0] - x_train.shape[0]}")

    # Train networks
    results = {}

    for n in config.neuron_counts:
        if config.verbose:
            print(f"\n  Training {n} neurons...")

        #---1. TRAINING: Returns the model and training results
        trained_model, training_result = train_network(
            SingleHiddenLayerNet(n, config.activation),
            x_train,
            y_train,
            config
        )
        #---2. EVALUATION: Evaluate on test set
        accuracy_results = evaluate_network(trained_model, x_plot, y_plot)

        #---3. COMBINE RESULTS
        results[n] = {**training_result, **accuracy_results}

        if config.verbose:
            status = "✓" if results[n]['converged'] else "✗"
            #Test MSE
            print(f"    {status} MSE = {results[n]['final_mse']:.6f}, Time = {results[n]['training_time']:.1f}s")

    # Compute metrics (final MSE of training set)
    metrics = compute_metrics(results, config.neuron_counts)

    # Plot
    save_dir = Path(config.save_dir) / config.target_function
    save_dir.mkdir(parents=True, exist_ok=True)
    save_path = save_dir / f"{config.activation}.png"

    plot_experiment_results(results, x_plot, y_plot, label, config, str(save_path))

    if config.verbose:
        print(f"\n  EXPERIMENT COMPLETE! N_min = {metrics['n_min']}, Best MSE = {metrics['best_mse']:.6f}")

    return results, metrics


# ============================================================================
# COMPARISON TABLE: print_comparison_table will generate a summary table
# comparing all activations across target functions, showing N_min, best MSE,
# and corresponding network width.
# ============================================================================

def print_comparison_table(all_results: Dict[str, Dict]):
    """
    Print comparison table across activations.

    Args:
        all_results: Dictionary mapping (function, activation) → (results, metrics)
    """
    print("\n" + "="*80)
    print("COMPARISON TABLE")
    print("="*80)

    print(f"\n{'Function':<15} {'Activation':<12} {'N_min':<10} {'Best MSE':<12} {'Best N':<10}")
    print("-" * 80)

    for (func, act), (results, metrics) in all_results.items():
        n_min = metrics['n_min'] if metrics['n_min'] else '>5000'
        print(f"{func:<15} {act:<12} {str(n_min):<10} {metrics['best_mse']:<12.6f} {metrics['best_n']:<10}")


if __name__ == "__main__":
    # Example usage
    config = ExperimentConfig(
        neuron_counts=[5,10],
        activation='tanh',
        learning_rate=0.8,
        target_function='sin',
        verbose=True,
        save_dir='results_try',
        epochs=3000
    )

    results, metrics = run_single_experiment(config)
    print(f"\nN_min: {metrics['n_min']}")
    print(f"Best MSE: {metrics['best_mse']:.6f} at N={metrics['best_n']}")


EXPERIMENT: sin + tanh
  Function: sin(x)
  Domain: [-3.0, 3.0]
  Samples: 1000
  Testing: [5, 10]
  Epochs: 3000, LR: 0.8
  Threshold: 0.01

  Train Samples: 1000, Test Samples: 1000

  Training 5 neurons...
      Epoch 1000/3000: Loss = 0.408293
      Epoch 2000/3000: Loss = 0.407103
      Epoch 3000/3000: Loss = 0.406879
    ✗ MSE = 0.394466, Time = 1.8s

  Training 10 neurons...
      Epoch 1000/3000: Loss = 0.010064
      Epoch 2000/3000: Loss = 0.123600
      Epoch 3000/3000: Loss = 0.070045
    ✗ MSE = 0.087839, Time = 1.8s
   Saved: results_try/sin/tanh.png

  EXPERIMENT COMPLETE! N_min = None, Best MSE = 0.087839

N_min: None
Best MSE: 0.087839 at N=10


##Run All code

### Complete experiment

In [ ]:
"""
Run All Experiments
===================
Execute complete experimental suite for UAT validation.

Usage:
    python run_experiments.py

This will generate all plots and save to results/ directory.
"""

#In case we are local, we need to import the previous code.
#from CoreComparison import ExperimentConfig, run_single_experiment, print_comparison_table
import itertools


def main():
    """Run all experiments."""

    print("="*80)
    print("UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION")
    print("="*80)
    print("""\nThis will run MxN experiments
      (M = number of target_functions and N= number of activations)""")

    # Define experimental grid
    target_functions = ['sin', 'gaussian', 'sin_high_freq']
    activations = ['relu', 'tanh', 'sigmoid']
    print("In our case, that is ",
          len(target_functions)*len(activations)," experiments")

    # Shared configuration
    base_config = {
        'neuron_counts': [1,10,50,100,500,1000],
        'epochs': 5000,
        'threshold': 0.01,
        'learning_rate': 0.01,
        'n_samples': 3000,
        'save_dir': 'results/complete',
        'verbose': True,
        'random_seed': 42,
    }

    # Run all combinations
    all_results = {}

    for func, act in itertools.product(target_functions, activations):
        config = ExperimentConfig(
            target_function=func,
            activation=act,
            **base_config
        )

        results, metrics = run_single_experiment(config)
        all_results[(func, act)] = (results, metrics)

    # Export experiment
    save_results(all_results)
    # Print comparison
    print_comparison_table(all_results)

    print("\n" + "="*80)
    print("ALL EXPERIMENTS COMPLETE!")
    print("="*80)
    print("\nResults saved in results/ directory:")
    print("  - results/sin/{relu,tanh,sigmoid}.png")
    print("  - results/sin_high_freq/{relu,tanh,sigmoid}.png")
    print("  - results/gaussian/{relu,tanh,sigmoid}.png")


if __name__ == "__main__":
    main()

UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION

This will run MxN experiments
      (M = number of target_functions and N= number of activations)
In our case, that is  9  experiments

EXPERIMENT: sin + relu
  Function: sin(x)
  Domain: [-3.0, 3.0]
  Samples: 3000
  Testing: [1, 10, 50, 100, 500, 1000]
  Epochs: 5000, LR: 0.01
  Threshold: 0.01

  Train Samples: 3000, Test Samples: 3000

  Training 1 neurons...
      Epoch 1000/5000: Loss = 0.465637
      Epoch 2000/5000: Loss = 0.465637
      Epoch 3000/5000: Loss = 0.465637
      Epoch 4000/5000: Loss = 0.465637
      Epoch 5000/5000: Loss = 0.465637
    ✗ MSE = 0.469203, Time = 2.9s

  Training 10 neurons...
      Epoch 1000/5000: Loss = 0.265488
      Epoch 2000/5000: Loss = 0.265887
      Epoch 3000/5000: Loss = 0.265354
      Epoch 4000/5000: Loss = 0.265339
      Epoch 5000/5000: Loss = 0.265334
    ✗ MSE = 0.273394, Time = 3.9s

  Training 50 neurons...
      Epoch 1000/5000: Loss = 0.090583
      Epoch 2000/5000: Loss

/tmp/ipython-input-2190664010.py:526: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout(rect=[0, 0, 1, 0.98])


   Saved: results/complete/sin/relu.png

  EXPERIMENT COMPLETE! N_min = 50, Best MSE = 0.000661

EXPERIMENT: sin + tanh
  Function: sin(x)
  Domain: [-3.0, 3.0]
  Samples: 3000
  Testing: [1, 10, 50, 100, 500, 1000]
  Epochs: 5000, LR: 0.01
  Threshold: 0.01

  Train Samples: 3000, Test Samples: 3000

  Training 1 neurons...
      Epoch 1000/5000: Loss = 0.453553
      Epoch 2000/5000: Loss = 0.451597
      Epoch 3000/5000: Loss = 0.450915
      Epoch 4000/5000: Loss = 0.450570
      Epoch 5000/5000: Loss = 0.450368
    ✗ MSE = 0.454901, Time = 3.1s

  Training 10 neurons...
      Epoch 1000/5000: Loss = 0.029382
      Epoch 2000/5000: Loss = 0.023685
      Epoch 3000/5000: Loss = 0.010405
      Epoch 4000/5000: Loss = 0.010279
      Epoch 5000/5000: Loss = 0.010235
    ✗ MSE = 0.011212, Time = 3.9s

  Training 50 neurons...
      Epoch 1000/5000: Loss = 0.002125
      Epoch 2000/5000: Loss = 0.000490
      Epoch 3000/5000: Loss = 0.000231
      Epoch 4000/5000: Loss = 0.000114
      E

/tmp/ipython-input-2190664010.py:526: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout(rect=[0, 0, 1, 0.98])


   Saved: results/complete/sin/sigmoid.png

  EXPERIMENT COMPLETE! N_min = 50, Best MSE = 0.000057

EXPERIMENT: gaussian + relu
  Function: exp(-x²)
  Domain: [-3.0, 3.0]
  Samples: 3000
  Testing: [1, 10, 50, 100, 500, 1000]
  Epochs: 5000, LR: 0.01
  Threshold: 0.01

  Train Samples: 3000, Test Samples: 3000

  Training 1 neurons...
      Epoch 1000/5000: Loss = 0.096640
      Epoch 2000/5000: Loss = 0.096640
      Epoch 3000/5000: Loss = 0.096640
      Epoch 4000/5000: Loss = 0.096640
      Epoch 5000/5000: Loss = 0.096640
    ✗ MSE = 0.095334, Time = 2.9s

  Training 10 neurons...
      Epoch 1000/5000: Loss = 0.000436
      Epoch 2000/5000: Loss = 0.000208
      Epoch 3000/5000: Loss = 0.000184
      Epoch 4000/5000: Loss = 0.000175
      Epoch 5000/5000: Loss = 0.000169
    ✓ MSE = 0.000170, Time = 3.4s

  Training 50 neurons...
      Epoch 1000/5000: Loss = 0.000006
      Epoch 2000/5000: Loss = 0.000003
      Epoch 3000/5000: Loss = 0.000003
      Epoch 4000/5000: Loss = 0.0000

### For 1, 10, 50 neurons

In [ ]:
"""
Run All Experiments
===================
Execute complete experimental suite for UAT validation.

Usage:
    python run_experiments.py

This will generate all plots and save to results/ directory.
"""

#In case we are local, we need to import the previous code.
#from CoreComparison import ExperimentConfig, run_single_experiment, print_comparison_table
import itertools


def main():
    """Run all experiments."""

    print("="*80)
    print("UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION")
    print("="*80)
    print("""\nThis will run MxN experiments
      (M = number of target_functions and N= number of activations)""")

    # Define experimental grid
    target_functions = ['sin', 'gaussian', 'sin_high_freq']
    activations = ['relu', 'tanh', 'sigmoid']
    print("In our case, that is ",
          len(target_functions)*len(activations)," experiments")

    # Shared configuration
    base_config = {
        'neuron_counts': [1,10,50],
        'epochs': 5000,
        'threshold': 0.01,
        'learning_rate': 0.01,
        'n_samples': 3000,
        'save_dir': 'results/final/1-10-50',
        'verbose': True,
        'random_seed': 42,
    }

    # Run all combinations
    all_results = {}

    for func, act in itertools.product(target_functions, activations):
        config = ExperimentConfig(
            target_function=func,
            activation=act,
            **base_config
        )

        results, metrics = run_single_experiment(config)
        all_results[(func, act)] = (results, metrics)

    # Export experiment
    save_results(all_results,"epochs-5000.csv")
    # Print comparison
    print_comparison_table(all_results)

    print("\n" + "="*80)
    print("ALL EXPERIMENTS COMPLETE!")
    print("="*80)
    print("\nResults saved in results/ directory:")
    print("  - results/sin/{relu,tanh,sigmoid}.png")
    print("  - results/sin_high_freq/{relu,tanh,sigmoid}.png")
    print("  - results/gaussian/{relu,tanh,sigmoid}.png")


if __name__ == "__main__":
    main()

UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION

This will run MxN experiments
      (M = number of target_functions and N= number of activations)
In our case, that is  9  experiments

EXPERIMENT: sin + relu
  Function: sin(x)
  Domain: [-3.0, 3.0]
  Samples: 3000
  Testing: [1, 10, 50]
  Epochs: 5000, LR: 0.01
  Threshold: 0.01

  Train Samples: 3000, Test Samples: 3000

  Training 1 neurons...
      Epoch 1000/5000: Loss = 0.465637
      Epoch 2000/5000: Loss = 0.465637
      Epoch 3000/5000: Loss = 0.465637
      Epoch 4000/5000: Loss = 0.465637
      Epoch 5000/5000: Loss = 0.465637
    ✗ MSE = 0.469203, Time = 2.9s

  Training 10 neurons...
      Epoch 1000/5000: Loss = 0.265488
      Epoch 2000/5000: Loss = 0.265887
      Epoch 3000/5000: Loss = 0.265354
      Epoch 4000/5000: Loss = 0.265339
      Epoch 5000/5000: Loss = 0.265334
    ✗ MSE = 0.273394, Time = 3.9s

  Training 50 neurons...
      Epoch 1000/5000: Loss = 0.090583
      Epoch 2000/5000: Loss = 0.090571
    

### For 100, 500, 1000 neurons

In [ ]:
"""
Run All Experiments
===================
Execute complete experimental suite for UAT validation.

Usage:
    python run_experiments.py

This will generate all plots and save to results/ directory.
"""

#In case we are local, we need to import the previous code.
#from CoreComparison import ExperimentConfig, run_single_experiment, print_comparison_table
import itertools


def main():
    """Run all experiments."""

    print("="*80)
    print("UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION")
    print("="*80)
    print("""\nThis will run MxN experiments
      (M = number of target_functions and N= number of activations)""")

    # Define experimental grid
    target_functions = ['sin']
    activations = ['relu', 'tanh', 'sigmoid']
    print("In our case, that is ",
          len(target_functions)*len(activations)," experiments")

    # Shared configuration
    base_config = {
        'neuron_counts': [100,500,1000],
        'epochs': 5000,
        'threshold': 0.01,
        'learning_rate': 0.01,
        'n_samples': 3000,
        'save_dir': 'results/final/100-500-1000',
        'verbose': True,
        'random_seed': 42
    }

    # Run all combinations
    all_results = {}

    for func, act in itertools.product(target_functions, activations):
        config = ExperimentConfig(
            target_function=func,
            activation=act,
            **base_config
        )

        results, metrics = run_single_experiment(config)
        all_results[(func, act)] = (results, metrics)

    #Export experiment
    save_results(all_results)
    # Print comparison
    print_comparison_table(all_results)

    print("\n" + "="*80)
    print("ALL EXPERIMENTS COMPLETE!")
    print("="*80)
    print("\nResults saved in results/ directory:")
    print("  - results/sin/{relu,tanh,sigmoid}.png")
    print("  - results/sin_high_freq/{relu,tanh,sigmoid}.png")
    print("  - results/gaussian/{relu,tanh,sigmoid}.png")


if __name__ == "__main__":
    main()

UNIVERSAL APPROXIMATION THEOREM: EXPERIMENTAL VALIDATION

This will run MxN experiments
      (M = number of target_functions and N= number of activations)
In our case, that is  3  experiments

EXPERIMENT: sin + relu
  Function: sin(x)
  Domain: [-3.0, 3.0]
  Samples: 3000
  Testing: [100, 500, 1000]
  Epochs: 5000, LR: 0.01
  Threshold: 0.01

  Train Samples: 3000, Test Samples: 3000

  Training 100 neurons...
      Epoch 1000/5000: Loss = 0.017437
      Epoch 2000/5000: Loss = 0.003929
      Epoch 3000/5000: Loss = 0.002650
      Epoch 4000/5000: Loss = 0.000948
      Epoch 5000/5000: Loss = 0.001994
    ✓ MSE = 0.002360, Time = 6.0s

  Training 500 neurons...
      Epoch 1000/5000: Loss = 0.001524
      Epoch 2000/5000: Loss = 0.001395
      Epoch 3000/5000: Loss = 0.005674
      Epoch 4000/5000: Loss = 0.002983
      Epoch 5000/5000: Loss = 0.000440
    ✓ MSE = 0.000478, Time = 26.9s

  Training 1000 neurons...
      Epoch 1000/5000: Loss = 0.001171
      Epoch 2000/5000: Loss = 0.